## **Load data**

In [ ]:
import pandas as pd
!gdown 1Pk3YtpgeaAF3EAEb5vZvzsz68SBITEwZ

Downloading...
From: https://drive.google.com/uc?id=1Pk3YtpgeaAF3EAEb5vZvzsz68SBITEwZ
To: /content/final_df.csv
100% 16.8M/16.8M [00:00<00:00, 93.5MB/s]


In [ ]:
final_df = pd.read_csv('final_df.csv')

In [ ]:
final_df.shape

(60514, 16)

In [ ]:
final_df.columns

Index(['paperId', 'questionId', 'candidateId', 'answerSheetId', 'answer_score',
       'maxScore', 'answeredCorrectly', 'Difficulty', 'firstSeenAt',
       'lastAnsweredAt', 'ResponseTime', 'Expected response time',
       'SequenceOrder', 'TotalTestingTime', 'IpAddress', 'ExamScore'],
      dtype='object')

# **1. Aberrance detection based on the difficulty of questions**

In [ ]:
# Load the data
data1 = final_df[['paperId', 'candidateId', 'Difficulty', 'answer_score', 'maxScore', 'answeredCorrectly', 'answerSheetId']]

# Group by answerSheetId and Difficulty
grouped = data1.groupby(['answerSheetId', 'Difficulty'])

# Calculate the correctness rate of each student at each difficulty level
difficulty_performance = grouped['answeredCorrectly'].mean().reset_index()
difficulty_performance.rename(columns={'answeredCorrectly': 'answeredCorrectly_mean'}, inplace=True)

# Merge back to the original data
data1 = data1.merge(difficulty_performance, on=['answerSheetId', 'Difficulty'])

# Adjust thresholds for high and low difficulty questions
high_difficulty_threshold = 0.8  # High difficulty level threshold
low_difficulty_threshold = 0.2   # Low difficulty level threshold

# Find students who performed well on high-difficulty questions
high_difficulty_good_performance = data1[(data1['Difficulty'] > high_difficulty_threshold) & (data1['answeredCorrectly'] == 1)]

# Find students who performed poorly on low-difficulty questions
low_difficulty_poor_performance = data1[(data1['Difficulty'] < low_difficulty_threshold) & (data1['answeredCorrectly'] == 0)]

# Find the intersection of the two sets of students
potential_cheaters_1 = high_difficulty_good_performance.merge(low_difficulty_poor_performance, on='answerSheetId', suffixes=('_high', '_low'))

# Get unique cheater answerSheetIds
answersheet_Ids_1 = potential_cheaters_1['answerSheetId'].unique()

# Get the potential cheaters' IDs and corresponding answerSheetIds
potential_cheaters = data1[data1['answerSheetId'].isin(answersheet_Ids_1)][['candidateId', 'answerSheetId']].drop_duplicates()

# Calculate the number of potential cheaters and the number of potential answerSheets
num_potential_cheaters = potential_cheaters['candidateId'].nunique()
num_potential_answersheets = len(answersheet_Ids_1)

# Calculate the total number of unique candidates and answerSheets
total_candidates = data1['candidateId'].nunique()
total_answersheets = data1['answerSheetId'].nunique()

# Calculate the percentage of potential cheaters and potential answerSheets
percentage_potential_cheaters = (num_potential_cheaters / total_candidates) * 100
percentage_potential_answersheets = (num_potential_answersheets / total_answersheets) * 100

# Print out the results
print(f"Number of potential cheaters: {num_potential_cheaters}")
print(f"Number of potential answerSheets: {num_potential_answersheets}")
print(f"Percentage of potential cheaters: {percentage_potential_cheaters:.2f}%")
print(f"Percentage of potential answerSheets: {percentage_potential_answersheets:.2f}%")

Number of potential cheaters: 140
Number of potential answerSheets: 140
Percentage of potential cheaters: 17.20%
Percentage of potential answerSheets: 13.86%


## **2. Aberrance detection based on the response time of questions**

In [ ]:
# Load the data
data2 = final_df[['paperId', 'questionId', 'candidateId', 'ResponseTime', 'Expected response time', 'Difficulty', 'answerSheetId']]

# Calculate the deviation of response time from expected response time
data2['response_time_deviation'] = data2['ResponseTime'] - data2['Expected response time']

# Group by questionId calculate mean and standard deviation of response time deviation
grouped = data2.groupby(['questionId'])

# Calculate mean and standard deviation of response time deviation for each question
response_time_stats = grouped['response_time_deviation'].agg(['mean', 'std']).reset_index()

# Merge stats back to original data
data2 = data2.merge(response_time_stats, on=['questionId'], suffixes=('', '_mean'))

# Define a stricter threshold for abnormal response times (4 standard deviations)
threshold = 4

# Identify potential cheaters with stricter threshold
data2['is_cheater'] = abs(data2['response_time_deviation']) > (threshold * data2['std'])

# Get unique cheater answerSheetIds
answersheet_Ids_2 = data2[data2['is_cheater']]['answerSheetId'].unique()

# Get the potential cheaters' IDs and corresponding answerSheetIds
potential_cheaters_2 = data2[data2['answerSheetId'].isin(answersheet_Ids_2)][['candidateId', 'answerSheetId']].drop_duplicates()

# Calculate the number of potential cheaters and the number of potential answerSheets
num_potential_cheaters_2 = potential_cheaters_2['candidateId'].nunique()
num_potential_answersheets_2 = len(answersheet_Ids_2)

# Calculate the total number of unique candidates and answerSheets
total_candidates_2 = data2['candidateId'].nunique()
total_answersheets_2 = data2['answerSheetId'].nunique()

# Calculate the percentage of potential cheaters and potential answerSheets
percentage_potential_cheaters_2 = (num_potential_cheaters_2 / total_candidates_2) * 100
percentage_potential_answersheets_2 = (num_potential_answersheets_2 / total_answersheets_2) * 100

# Print out the results
print(answersheet_Ids_1)
print(potential_cheaters)
print(f"Number of potential cheaters: {num_potential_cheaters_2}")
print(f"Number of potential answerSheets: {num_potential_answersheets_2}")
print(f"Percentage of potential cheaters: {percentage_potential_cheaters_2:.2f}%")
print(f"Percentage of potential answerSheets: {percentage_potential_answersheets_2:.2f}%")

['6bbfed12-f0f6-4398-8afd-d458341fba3f'
 '7347e573-9cf6-41ca-97a7-6fb979939f5b'
 'aa93b23a-ed23-496d-905a-f7fced181d8c'
 '3da10398-b038-4eee-9247-523853c67cd6'
 '24ba9117-7c1f-42e4-b3ff-73244c7e863f'
 'c2cb8c59-e3d2-464c-84ba-77d7050aea1d'
 'd6257b1c-4e59-47e3-b4ed-d615a4f58bea'
 '420e94a7-4700-4dba-ac73-be6d1412db7e'
 '3b4b2223-8acb-44fe-8e75-1e9380a094fc'
 '9a3c47f1-22ad-474c-bc74-b8ebe64f7493'
 'dcdbc3b2-0e7c-46c5-ab5e-b23d13bc21b0'
 '1a7b5864-c1ea-48c7-8195-a4472af946f6'
 '82493630-d9bd-4092-85f7-5e765d45992e'
 'd7a6779f-c922-4dbc-ba61-70e1498e131b'
 '31124455-b96f-44fa-a133-5130268b492e'
 '7408f5c4-6f57-4bd6-a81e-214000148b1f'
 '88345f3a-18b0-43f1-bf73-c77383fb712e'
 '7868c97b-6590-43fa-9acd-b42ef8203f04'
 '970fb573-7e19-40a6-8137-62f358032cb8'
 'df227cea-1dfc-4b6f-9108-b7d24ecdfd35'
 'f72b1e2c-7300-4742-a9b6-811232aaacce'
 '7a9302b6-21f3-4a29-9ff3-a22e4e0e4ef2'
 '4bdc6936-1779-4b18-8865-a31e80ed9a16'
 '67e9f6a6-9ea0-4fd6-a235-4dd23cebd7a3'
 'baeafddf-cf76-4a30-827a-b7dd61059738'


<ipython-input-7-c7f7a4d7729f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['response_time_deviation'] = data2['ResponseTime'] - data2['Expected response time']


### **3. Cosine Similarity-Based Collusion Detection**

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

# Load the data
data3 = final_df

# Select relevant variables
data3 = data3[['paperId', 'questionId', 'candidateId', 'answeredCorrectly', 'answerSheetId']]

# Create a pivot table with students as rows and questions as columns
pivot_table = data3.pivot_table(
    index='answerSheetId', columns='questionId', values='answeredCorrectly', fill_value=0
)

# Compute pairwise cosine similarity between students
similarity_matrix = 1 - squareform(pdist(pivot_table, metric='cosine'))

# Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=pivot_table.index, columns=pivot_table.index)

# Define a similarity threshold for identifying potential collusion
similarity_threshold = 0.9  # Adjust this threshold based on your specific needs

# Find pairs of students with high similarity (potential collusion)
collusion_pairs = np.where(similarity_matrix > similarity_threshold)

# Filter out self-comparisons and remove duplicate pairs (e.g., (A, B) and (B, A))
collusion_pairs = [(pivot_table.index[i], pivot_table.index[j]) for i, j in zip(*collusion_pairs) if i < j]

# Print results
print(f"Potential collusion pairs: {collusion_pairs}")
print(f"Number of potential collusion pairs: {len(collusion_pairs)}")

# Get unique cheater answerSheet IDs
answersheet_Ids_3 = set([pair[0] for pair in collusion_pairs] + [pair[1] for pair in collusion_pairs])

# Get the potential cheaters' IDs and corresponding answerSheetIds
potential_cheaters_3 = data3[data3['answerSheetId'].isin(answersheet_Ids_3)][['candidateId', 'answerSheetId']].drop_duplicates()

# Calculate the number of potential cheaters and the number of potential answerSheets
num_potential_cheaters_3 = potential_cheaters_3['candidateId'].nunique()
num_potential_answersheets_3 = len(answersheet_Ids_3)

# Calculate the total number of unique candidates and answerSheets
total_candidates_3 = data3['candidateId'].nunique()
total_answersheets_3 = data3['answerSheetId'].nunique()

# Calculate the percentage of potential cheaters and potential answerSheets
percentage_potential_cheaters_3 = (num_potential_cheaters_3 / total_candidates_3) * 100
percentage_potential_answersheets_3 = (num_potential_answersheets_3 / total_answersheets_3) * 100

# Print out the results
print(f"Number of potential cheaters: {num_potential_cheaters_3}")
print(f"Number of potential answerSheets: {num_potential_answersheets_3}")
print(f"Percentage of potential cheaters: {percentage_potential_cheaters_3:.2f}%")
print(f"Percentage of potential answerSheets: {percentage_potential_answersheets_3:.2f}%")

Potential collusion pairs: [('01c09d9d-d16e-4ae1-ac90-3189207f745e', '105ae3da-f1fd-4159-94f2-338934f4e36d'), ('0396e6f3-8775-4640-ac97-1167246d8314', '400c33c1-c3ec-4e22-b717-045f53aa81b9'), ('0396e6f3-8775-4640-ac97-1167246d8314', '89de5e5e-f4bd-452e-a65e-5068776e5e2e'), ('046ba4fa-9ef0-43e5-934d-319b7b2e779d', 'e95c18be-d919-466e-a59f-7b8a9a581a2c'), ('05790ea4-3506-485b-957e-ab20c405a069', '07ff10b2-dab5-49f0-8810-64bafb6d8876'), ('05790ea4-3506-485b-957e-ab20c405a069', '5593f230-7a51-4f0a-a756-84f0a8a83658'), ('05790ea4-3506-485b-957e-ab20c405a069', 'b65ddf5a-d59a-4f31-9277-2ab6c41c4174'), ('05790ea4-3506-485b-957e-ab20c405a069', 'f0ffda12-9e05-48df-838b-028e542feb6d'), ('059cfd8c-c058-46b0-bbdb-54818e39af6d', '2e19d544-aef5-4599-a944-5fb6a63e2d57'), ('05b58f13-6dda-422c-a8b5-d37302fdf58d', '50062d12-dcd8-4be1-aff0-197882e68549'), ('05b58f13-6dda-422c-a8b5-d37302fdf58d', '597713f2-fb95-429d-bf37-32b10ec24d31'), ('05b58f13-6dda-422c-a8b5-d37302fdf58d', 'ba8f4e61-8d96-4ca4-8e24-6128

## **5.Kullback-Leibler Divergence Analysis**

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import pairwise_distances
from scipy.stats import entropy
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import entropy

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

def calculate_kl_divergences(df):
    # Drop all NaN in ResponseTime column
    df = df.dropna(subset=['ResponseTime'])

    # Calculate average response time per question for the group
    group_response_times = df.groupby('questionId')['ResponseTime'].mean()

    # Define a function to calculate KL Divergence
    def calculate_kl_divergence(individual_response_times, group_response_times):
        # Normalize the distributions
        individual_sum = individual_response_times.sum()
        group_sum = group_response_times.sum()

        if individual_sum == 0 or group_sum == 0:
            return np.nan

        individual_distribution = individual_response_times / individual_sum
        group_distribution = group_response_times / group_sum

        # Add a small value to avoid division by zero in log
        epsilon = 1e-10
        kl_divergence = entropy(individual_distribution + epsilon, group_distribution + epsilon)
        return kl_divergence

    # Prepare a DataFrame to store KL Divergence results
    kl_divergence_results = []

    # Iterate over each answersheet
    for answersheet_id in df['answerSheetId'].unique():
        # Get response times for the answersheet
        candidate_data = df[df['answerSheetId'] == answersheet_id]
        candidate_response_times = candidate_data.groupby('questionId')['ResponseTime'].sum()

        # Align the distributions to the same questions
        aligned_group_times = group_response_times.loc[candidate_response_times.index]

        # Ensure both distributions have the same length and no zero values
        if len(candidate_response_times) == len(aligned_group_times) and all(candidate_response_times > 0) and all(aligned_group_times > 0):
            # Calculate KL Divergence for the candidate
            kl_divergence = calculate_kl_divergence(candidate_response_times, aligned_group_times)
        else:
            kl_divergence = np.nan

        # Store the result
        kl_divergence_results.append({
            'answerSheetId': answersheet_id,
            'candidateId': candidate_data['candidateId'].iloc[0],
            'KL_Divergence': kl_divergence
        })

    # Convert results to DataFrame
    kl_divergence_df = pd.DataFrame(kl_divergence_results)
    return kl_divergence_df

# Use the DataFrame to calculate KL Divergence
kl_divergence_df = calculate_kl_divergences(final_df)

# Flag candidates with high KL Divergence
threshold = kl_divergence_df['KL_Divergence'].quantile(0.99)  # For example, top 1% as threshold
potential_cheaters = kl_divergence_df[kl_divergence_df['KL_Divergence'] > threshold]

# Calculate the percentage of flagged candidates and answer sheets
total_candidates = len(final_df['candidateId'].unique())
total_answer_sheets = len(final_df['answerSheetId'].unique())

num_potential_cheaters = len(potential_cheaters['candidateId'].unique())
num_potential_answer_sheets = len(potential_cheaters)

percentage_potential_cheaters = (num_potential_cheaters / total_candidates) * 100
percentage_potential_answer_sheets = (num_potential_answer_sheets / total_answer_sheets) * 100

# Get IDs of potential cheaters
answersheet_Ids_5 = potential_cheaters['answerSheetId'].tolist()

# Print out the results
print(f"Number of potential cheaters: {num_potential_cheaters}")
print(f"Number of potential answerSheets: {num_potential_answer_sheets}")
print(f"Percentage of potential cheaters: {percentage_potential_cheaters:.2f}%")
print(f"Percentage of potential answerSheets: {percentage_potential_answer_sheets:.2f}%")

Number of potential cheaters: 11
Number of potential answerSheets: 11
Percentage of potential cheaters: 1.35%
Percentage of potential answerSheets: 1.09%


## **Find common potential cheaters**

## **Identify answersheets flagged by all methods**

In [ ]:
# Find the intersection of all methods
intersection_answersheet_ids = set(answersheet_Ids_1) & set(answersheet_Ids_2) & set(answersheet_Ids_3)  & set(answersheet_Ids_5)

# Get the final set of cheaters' candidate IDs and answersheet IDs
final_cheaters = final_df[final_df['answerSheetId'].isin(intersection_answersheet_ids)][['candidateId', 'answerSheetId']].drop_duplicates()

# Calculate statistics for the final set of cheaters
num_final_cheaters = final_cheaters['candidateId'].nunique()
num_final_answersheets = len(intersection_answersheet_ids)
total_candidates = final_df['candidateId'].nunique()

percentage_final_cheaters = (num_final_cheaters / total_candidates) * 100
percentage_final_answersheets = (num_final_answersheets / total_answersheets) * 100

# Print out the final results
print("Final Set of Cheaters' IDs and Corresponding answerSheetIds:")
print(final_cheaters)
print(f"Number of final cheaters: {num_final_cheaters}")
print(f"Number of final answerSheets: {num_final_answersheets}")
print(f"Percentage of final cheaters: {percentage_final_cheaters:.2f}%")
print(f"Percentage of final answerSheets: {percentage_final_answersheets:.2f}%")


Final Set of Cheaters' IDs and Corresponding answerSheetIds:
Empty DataFrame
Columns: [candidateId, answerSheetId]
Index: []
Number of final cheaters: 0
Number of final answerSheets: 0
Percentage of final cheaters: 0.00%
Percentage of final answerSheets: 0.00%


## **Identify answersheets flagged by at least 4 methods**

In [ ]:
# Convert all answersheetIds lists to standard Python lists
answersheetIds_1 = list(answersheet_Ids_1)
answersheetIds_2 = list(answersheet_Ids_2)
answersheetIds_3 = list(answersheet_Ids_3)
answersheetIds_5 = list(answersheet_Ids_5)

# Get the unique answersheetIds from all methods
all_unique_answersheetIds = list(set(answersheetIds_1 + answersheetIds_2 + answersheetIds_3 + answersheetIds_5))

# Combine all answersheetIds into a single DataFrame
all_cheaters_df = pd.DataFrame({
    'answerSheetId': all_unique_answersheetIds,
    'flagged_by_1': [1 if i in answersheetIds_1 else 0 for i in all_unique_answersheetIds],
    'flagged_by_2': [1 if i in answersheetIds_2 else 0 for i in all_unique_answersheetIds],
    'flagged_by_3': [1 if i in answersheetIds_3 else 0 for i in all_unique_answersheetIds],
    'flagged_by_5': [1 if i in answersheetIds_5 else 0 for i in all_unique_answersheetIds]
})

# Calculate how many times each answersheet was flagged
all_cheaters_df['total_flags'] = all_cheaters_df[['flagged_by_1', 'flagged_by_2', 'flagged_by_3',  'flagged_by_5']].sum(axis=1)

# Identify answersheets flagged by at least 4 methods
multi_flagged_cheaters = all_cheaters_df[all_cheaters_df['total_flags'] >= 3]

# Get the final set of cheaters' candidateIds and answersheetIds
final_cheaters_multi = final_df[final_df['answerSheetId'].isin(multi_flagged_cheaters['answerSheetId'])][['candidateId', 'answerSheetId']].drop_duplicates()

# Calculate statistics for the final set of cheaters flagged by multiple methods
num_final_cheaters_multi = final_cheaters_multi['candidateId'].nunique()
num_final_answersheets_multi = len(multi_flagged_cheaters)
total_candidates = final_df['candidateId'].nunique()

percentage_final_cheaters_multi = (num_final_cheaters_multi / total_candidates) * 100
percentage_final_answersheets_multi = (num_final_answersheets_multi / total_answersheets) * 100

# Print out the final results from the multiple method analysis
print("Final Set of Cheaters' IDs and Corresponding answersheetIds (Flagged by Multiple Methods):")
print(final_cheaters_multi)
print(f"Number of final cheaters: {num_final_cheaters_multi}")
print(f"Number of final answersheets: {num_final_answersheets_multi}")
print(f"Percentage of final cheaters: {percentage_final_cheaters_multi:.2f}%")
print(f"Percentage of final answersheets: {percentage_final_answersheets_multi:.2f}%")

Final Set of Cheaters' IDs and Corresponding answersheetIds (Flagged by Multiple Methods):
                                candidateId  \
12584  3602fa4e-08b0-4db2-83fa-e81a97a68f23   
16967  4ad0d818-c2df-4d3b-8fbb-4cd78d369681   
23459  65737f9a-3a4e-4689-92a2-bb918c45b63b   
35383  96c3e625-a937-4c74-a837-038695a73927   
35761  97c35ee0-131a-4229-a488-a9e02475b574   
36360  9b725949-0c8b-479f-a178-7d1d7f2ca010   
41758  b25edf39-bc91-4a98-a13c-6f66e3a720e8   
45645  bfaf7fe9-c6d5-4fe5-8cd3-20882e574237   

                              answerSheetId  
12584  969af726-bf5a-4621-bafd-3c33237bdf7b  
16967  10e65d13-0806-4fec-8d14-2a51b858b31e  
23459  d1dcbd25-b224-490b-8425-f1721448a325  
35383  0a23ca23-b40c-4836-a81f-fd7faf3feb45  
35761  ef923be0-c032-4f23-8425-5fc94224537b  
36360  3b31eba2-cc30-42a3-8389-a9cf38a0ab2d  
41758  2eb8545b-9812-4368-8c8c-66f896f43652  
45645  f64c050c-3ec3-46d6-8795-ed03cc907343  
Number of final cheaters: 8
Number of final answersheets: 8
Percentage 

In [ ]:
# Convert answersheetIds to candidateIds for each method using final_df
candidates_method_1 = final_df[final_df['answerSheetId'].isin(answersheetIds_1)][['candidateId', 'answerSheetId']]
candidates_method_2 = final_df[final_df['answerSheetId'].isin(answersheetIds_2)][['candidateId', 'answerSheetId']]
candidates_method_3 = final_df[final_df['answerSheetId'].isin(answersheetIds_3)][['candidateId', 'answerSheetId']]
candidates_method_5 = final_df[final_df['answerSheetId'].isin(answersheetIds_5)][['candidateId', 'answerSheetId']]

# Print each method's candidateIds and answersheetIds
print("Candidates flagged by method 1:")
print(candidates_method_1)
print("Candidates flagged by method 2:")
print(candidates_method_2)
print("Candidates flagged by method 3:")
print(candidates_method_3)
print("Candidates flagged by method 5:")
print(candidates_method_5)

# Print each method's answersheetIds
print("Answersheet IDs flagged by method 1:")
print(answersheetIds_1)
print("Answersheet IDs flagged by method 2:")
print(answersheetIds_2)
print("Answersheet IDs flagged by method 3:")
print(answersheetIds_3)
print("Answersheet IDs flagged by method 5:")
print(answersheetIds_5)

Candidates flagged by method 1:
                                candidateId  \
0      0005c95f-814b-4117-8065-f25c33f1e712   
1      0005c95f-814b-4117-8065-f25c33f1e712   
2      0005c95f-814b-4117-8065-f25c33f1e712   
3      0005c95f-814b-4117-8065-f25c33f1e712   
4      0005c95f-814b-4117-8065-f25c33f1e712   
...                                     ...   
60509  ffff87dd-42c5-4479-acc7-acb6f1e45b17   
60510  ffff87dd-42c5-4479-acc7-acb6f1e45b17   
60511  ffff87dd-42c5-4479-acc7-acb6f1e45b17   
60512  ffff87dd-42c5-4479-acc7-acb6f1e45b17   
60513  ffff87dd-42c5-4479-acc7-acb6f1e45b17   

                              answerSheetId  
0      6bbfed12-f0f6-4398-8afd-d458341fba3f  
1      6bbfed12-f0f6-4398-8afd-d458341fba3f  
2      6bbfed12-f0f6-4398-8afd-d458341fba3f  
3      6bbfed12-f0f6-4398-8afd-d458341fba3f  
4      6bbfed12-f0f6-4398-8afd-d458341fba3f  
...                                     ...  
60509  6fb8974b-d87b-4470-9edd-0e14b1c5a350  
60510  6fb8974b-d87b-4470-9edd-0e14

In [ ]:
# Method 1
candidates_method_1 = final_df[final_df['answerSheetId'].isin(answersheetIds_1)][['candidateId', 'answerSheetId']]
candidates_method_1.to_csv('candidates_method_1.csv', index=False)

# Method 2
candidates_method_2 = final_df[final_df['answerSheetId'].isin(answersheetIds_2)][['candidateId', 'answerSheetId']]
candidates_method_2.to_csv('candidates_method_2.csv', index=False)

# Method 3
candidates_method_3 = final_df[final_df['answerSheetId'].isin(answersheetIds_3)][['candidateId', 'answerSheetId']]
candidates_method_3.to_csv('candidates_method_3.csv', index=False)

# Method 5
candidates_method_5 = final_df[final_df['answerSheetId'].isin(answersheetIds_5)][['candidateId', 'answerSheetId']]
candidates_method_5.to_csv('candidates_method_5.csv', index=False)
